In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [22]:
import pandas as pd

# 1) Read in a few rows to inspect exact names
df = pd.read_csv('data/raw/combined_price_data.csv', nrows=5)
print("Columns:", df.columns.tolist())

# 2) Reload full data and standardize column names
df = pd.read_csv('data/raw/combined_price_data.csv')
df.columns = df.columns.str.strip().str.lower()   # e.g. 'Date ' → 'date', 'Close' → 'close'

# 3) Parse the date column (now guaranteed to be 'date')
df['timestamp'] = pd.to_datetime(df['date'])

# 4) Rename and set index
df = df.rename(columns={'close': 'price', 'volume': 'volume'})  # 'volume' stays the same
df = df.set_index('timestamp')

# 5) Verify
print(df[['price','volume']].head())
print(df.info())


Columns: ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker', 'Adj Close.1', 'Close.1', 'High.1', 'Low.1', 'Open.1', 'Volume.1', 'Adj Close.2', 'Close.2', 'High.2', 'Low.2', 'Open.2', 'Volume.2']
                         price       volume
timestamp                                  
NaT                       AAPL         AAPL
2015-01-02  27.332500457763672  212818400.0
2015-01-05             26.5625  257142000.0
2015-01-06  26.565000534057617  263188400.0
2015-01-07             26.9375  160423600.0
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7894 entries, NaT to 2025-06-18
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         7893 non-null   object
 1   adj close    2632 non-null   object
 2   price        2632 non-null   object
 3   high         2632 non-null   object
 4   low          2632 non-null   object
 5   open         2632 non-null   object
 6   volume       2632 non-nu

In [23]:
df = df.rename(columns={
    'Date':   'timestamp',
    'Close':  'price',
    'Volume': 'volume'
    # if you have 'Open', 'High', 'Low' you can include them too
})
df = df.set_index('timestamp')

print(df[['price', 'volume']].head())
print(df.info())

KeyError: "None of ['timestamp'] are in the columns"

In [17]:
print(df.columns.tolist())

['Adj Close', 'price', 'High', 'Low', 'Open', 'volume', 'Ticker', 'Adj Close.1', 'Close.1', 'High.1', 'Low.1', 'Open.1', 'Volume.1', 'Adj Close.2', 'Close.2', 'High.2', 'Low.2', 'Open.2', 'Volume.2']


In [14]:
# Cell 3a: drop duplicates
df = df.drop_duplicates()

# Cell 3b: handle missing values
# option A: drop any row with critical nulls
df = df.dropna(subset=['Close', 'Volume'])
# option B: fill small gaps
df['Close'] = df['Close'].ffill()

# Cell 3c: timezone & index
df['Date'] = df['Date'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
df = df.set_index('Date')

/var/folders/18/wgg8_l9s7q36pfj8d5y7hpzh0000gn/T/ipykernel_34528/1283499662.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_1min = df.resample('1T').agg({


In [ ]:
# Cell 5a: simple returns
df_1min['return'] = df_1min['price'].pct_change()

# Cell 5b: rolling stats
df_1min['ma_10']    = df_1min['price'].rolling(10).mean()
df_1min['vol_10']   = df_1min['return'].rolling(10).std()

# Cell 5c: VWAP (assumes you have 'vwap_price' or use price*volume)
df_1min['vwap'] = (df_1min['price'] * df_1min['volume']).rolling(10).sum() \
                  / df_1min['volume'].rolling(10).sum()